# Manual Correct Ilastik Segmentation
Here we wil use Napari to manually correct segmentation.
See documentation [here](https://napari.org/howtos/layers/labels.html)

Note that the keyboard shortcuts in the documentations are wrong!  
See below for details. 

In [1]:

#next line is required for Napari
%gui qt

#main data analysis packages
import numpy as np

#image viewer
import napari
from napari.utils.notebook_display import nbscreenshot

#out of memory computation
from dask_image.imread import imread
import dask.array as da

#path handling
import pathlib

#file handling
import h5py

#Instead of dask_image.imread.imread() you can also use tifffile.imread() to directly read images into memory
#import tifffile

In [2]:
#we initiate a cashe for Dask to speed up repeated computation (important for working with Napari)
from dask.cache import Cache
cache = Cache(2e9)  # Leverage two gigabytes of memory
cache.register()    # Turn cache on globally

---

## Import data

In [3]:
#Set the path to the folder that contains project data
root = pathlib.Path(pathlib.Path.home(), 'Andreas', 'Training')

#raw images
im_name0 = 'ph_training.tif' #set name of image
im_name1 = 'gfp_training.tif' #set name of image

#segmentation output
seg_file_name = root / 'processed_labels.hdf5'
seg_file_name_corr = root / 'processed_labels_corrected.hdf5'


im0_stack = imread(root / im_name0) #load image with dask-image for out of memory processing 
im1_stack = imread(root / im_name1) #load image with dask-image for out of memory processing 

To visualize the data we will use [Napari](https://napari.org), that allows for interactive image visualization. 
You can use the slider at the bottom of the window to scroll trough time. 

In [10]:
#setup napari viewer, with 2 channel image
viewer = napari.view_image(im0_stack, name="phase", colormap="gray")
viewer.add_image(im1_stack, name="rfp", colormap="red", opacity=0.5)
napari.run()

---
## Manual Correction

First we initialize manual correction or load existing file from disk

In [5]:

#load processed segmentation if possible
if not seg_file_name_corr.exists():
    #load original segmentation
    with h5py.File(seg_file_name, 'r') as f:
        watershed_labels = np.array(f['raw_labels'])
    
    #initialize new corrected layer
    print('Initialize manual correction')
    frame_status = np.full(watershed_labels.shape[0], -1)
    corrected_labels = watershed_labels.copy()
else:
    #load existing correction file
    print('Load manual correction')
    with h5py.File(seg_file_name_corr, 'r') as f:
        frame_status = np.array(f['frame_status'])
        corrected_labels = np.array(f['corrected_labels'])   

Load manual correction


In [11]:
np_corrected_labels = viewer.add_labels(corrected_labels, name='processed')
np_corrected_labels.visible = True 

You can use the default key bindings:

- 1 = eraser
- 2 = brush
- 3 = paint-bucket
- 4 = color picker
- 5 = zoom/pan mode
- M = select new color label (that is not yet in use)
- arrow left/right: switch frames

In addition we setup some new key bindings:
- a = mark frame as corrected, store data and go to next frame
- d = mark frame to discard later, store data and go to next frame
- n = go to first unchecked frame
- ` = toggle label layer visibility
- b = save data
- 8 = set brush to small size (1 pixel)
- 9 = set brush to medium size
- 0 = set brush to large size
- [ = decrease brush size
- ] = increase brush size

Now go through each frame, if possible correct segmentation and press `a` when done to make frame as corrected.  
If segmentation/image quality is too bad press `d` to mark frame to be discarded at end.

In [12]:

def save_corrected_segementation():
    with h5py.File(seg_file_name_corr, 'w') as f:
        f.create_dataset('corrected_labels', data=corrected_labels)
        f.create_dataset('frame_status', data=frame_status)
    print('data saved')

@viewer.bind_key('n', overwrite=True)
def next_unprocessed_frame(viewer):
    next_fr = np.where(frame_status == -1)[0][0]
    viewer.dims.current_step = (next_fr, *viewer.dims.current_step[1:])

@viewer.bind_key('a', overwrite=True)
def skip_frame(viewer):
    frame_status[viewer.dims.current_step[0]] = 1
    print('approved frame %i' % viewer.dims.current_step[0])
    save_corrected_segementation()
    viewer.dims.current_step = (viewer.dims.current_step[0]+1, *viewer.dims.current_step[1:])
    
@viewer.bind_key('d', overwrite=True)
def skip_frame(viewer):
    frame_status[viewer.dims.current_step[0]] = 0 
    print('discard frame %i' % viewer.dims.current_step[0])
    save_corrected_segementation()
    viewer.dims.current_step = (viewer.dims.current_step[0]+1, *viewer.dims.current_step[1:])
        
@viewer.bind_key('`', overwrite=True)
def skip_frame(viewer):
    np_corrected_labels.visible = not np_corrected_labels.visible   
    
@viewer.bind_key('b', overwrite=True)    
def save_data(viewer):
    save_corrected_segementation()
    
@viewer.bind_key('8', overwrite=True)    
def set_brush_small(viewer):
    np_corrected_labels.brush_size = 1

@viewer.bind_key('9', overwrite=True)    
def set_brush_med(viewer):
    np_corrected_labels.brush_size = 5
    
@viewer.bind_key('0', overwrite=True)    
def set_brush_large(viewer):
    np_corrected_labels.brush_size = 20  
   
@viewer.bind_key(']', overwrite=True)    
def increase_brush(viewer):
    np_corrected_labels.brush_size += 1     
    
@viewer.bind_key('[', overwrite=True)    
def decrease_brush(viewer):
    np_corrected_labels.brush_size -= 1  
    np_corrected_labels.brush_size = max(np_corrected_labels.brush_size, 1)
 
       

approved frame 114
data saved
discard frame 115
data saved
approved frame 116
data saved
approved frame 117
data saved


Traceback (most recent call last):
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 512, in mouseReleaseEvent
    self._vispy_mouse_release(
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/app/base.py", line 224, in _vispy_mouse_release
    ev = self._vispy_canvas.events.mouse_release(**kwargs)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/na

approved frame 118
data saved
approved frame 119
data saved
approved frame 120
data saved


Traceback (most recent call last):
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 512, in mouseReleaseEvent
    self._vispy_mouse_release(
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/app/base.py", line 224, in _vispy_mouse_release
    ev = self._vispy_canvas.events.mouse_release(**kwargs)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/na

approved frame 121
data saved
discard frame 122
data saved
discard frame 123
data saved
approved frame 124
data saved
approved frame 125
data saved
approved frame 126
data saved
discard frame 127
data saved
approved frame 128
data saved
approved frame 129
data saved


...

Traceback (most recent call last):
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 512, in mouseReleaseEvent
    self._vispy_mouse_release(
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/app/base.py", line 224, in _vispy_mouse_release
    ev = self._vispy_canvas.events.mouse_release(**kwargs)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/na

approved frame 137
data saved
approved frame 138
data saved


Traceback (most recent call last):
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 512, in mouseReleaseEvent
    self._vispy_mouse_release(
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/app/base.py", line 224, in _vispy_mouse_release
    ev = self._vispy_canvas.events.mouse_release(**kwargs)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/na

approved frame 139
data saved


Traceback (most recent call last):
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 561, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/Users/simonvanvliet/opt/miniconda3/envs/i2i_env/li

In [13]:
processed_file = root /  'processed_labels_fully_corrected_2022-06-15.hdf5'

with h5py.File(processed_file, 'w') as f:
    f.create_dataset('corrected_labels', data=corrected_labels)
    f.create_dataset('frame_status', data=frame_status)

In [20]:
training_im = corrected_labels[frame_status==1,:,:]
training_im.shape

(114, 1024, 1024)

In [21]:
approved_labels = corrected_labels[frame_status==1,:,:]
approved_labels.shape

(114, 1024, 1024)

In [22]:
processed_file = root /  'training_date.hdf5'

with h5py.File(processed_file, 'w') as f:
    f.create_dataset('labels', data=approved_labels)
    f.create_dataset('phase', data=training_im)